# Import text from the JSON file for SNA

Go through all comments (and comments of comments) and convert each post into a row of the final dataframe.

In [2]:
# load libraries
import os
import json
import pandas as pd
# import numpy as np

## 1, Create the column names for the output dataframe to store post information extracted from the .json files.

This cell reads the element names in CSMM101_3T2020_discussion_threads.json as the column names.

We also add and remove some columns.

In [3]:
with open("./CSMM101_3T2020_discussion_threads.json") as ft:
        discussions_t = json.load(ft)

ft.close()

# Create a dataframe (df_SNA) to store data for every post, comment, and answer.
df_col_names = [str(element) for element in list(discussions_t[0].keys())]
df_col_names += ['user_from']
df_col_names += ['user_from_role']
df_col_names += ['user_from_email']
df_col_names += ['user_to']
df_col_names += ['user_to_role']
df_col_names += ['user_to_email']
df_col_names.remove('user')
df_col_names.remove('document')
df_col_names.remove('comments')

print(df_col_names)

['url', 'type', 'title', 'category', 'subcategory', 'votes', 'private', 'created_at', 'text', 'user_from', 'user_from_role', 'user_from_email', 'user_to', 'user_to_role', 'user_to_email']


## 2, Check the .csv files that store user information: numbers of emails, user ids, and user names should be the same.

Also, please note that the Username in gradebook and user name in json file do not match. So we use email address to
link the two dataset.

In [4]:
def check_users(user_file):
    user_csv = pd.read_csv(user_file, sep = ',')
    uni_emails = len(user_csv['Email'].unique())
    uni_stuid = len(user_csv['Student ID'].unique())
    uni_names = len(user_csv['Username'].unique())

    if uni_emails == uni_stuid & uni_stuid == uni_names :
        print("Numbers of Emails, Student IDs, and Names all match.\n")
    else :
        print(uni_emails, uni_stuid, uni_names)

data_dir = './Beta1_Data/'
for fname in os.listdir(data_dir) :
    if fname.endswith(".csv"):
        print(os.path.join(data_dir, fname))
        check_users(os.path.join(data_dir, fname))

./Beta1_Data/ColumbiaX_CSMM103x_3T2019_grade_report_2019-12-29-2126.csv
Numbers of Emails, Student IDs, and Names all match.

./Beta1_Data/ColumbiaX_BAMM103x_3T2019_grade_report_2019-12-29-2121.csv
Numbers of Emails, Student IDs, and Names all match.

./Beta1_Data/ColumbiaX_CSMM101x_3T2019_grade_report_2019-12-29-0248.csv
Numbers of Emails, Student IDs, and Names all match.

./Beta1_Data/ColumbiaX_BAMM104x_3T2019_grade_report_2019-12-29-2127.csv
5967 5967 5966
./Beta1_Data/ColumbiaX_CSMM104x_3T2019_grade_report_2019-12-29-2119.csv
Numbers of Emails, Student IDs, and Names all match.

./Beta1_Data/ColumbiaX_BAMM102x_3T2019_grade_report_2019-12-29-2122.csv
Numbers of Emails, Student IDs, and Names all match.

./Beta1_Data/ColumbiaX_CSMM102x_3T2019_grade_report_2019-12-29-2126.csv
Numbers of Emails, Student IDs, and Names all match.

./Beta1_Data/ColumbiaX_BAMM101x_3T2019_grade_report_2019-12-29-2124.csv
Numbers of Emails, Student IDs, and Names all match.



## 3, Define json_to_csv function. Input is the json file and output is the dataframe of the flattened records.

In [5]:
def json_to_csv(json_f):

    # read json file
    with open(str(data_dir + json_f)) as f:
        discussions = json.load(f)

    f.close()

    result = pd.DataFrame([], columns = df_col_names)

    for discussion in discussions:
        user = {'name': None, 'role': None, 'email': None}
        result = read_comment(discussion, result, user, None)

    return result

## 4, Define read_comment function, which converts each post/comment to a row in the output dataframe.

In [6]:
# read_comment is a recursive function to read all comments (and answers)
def read_comment(comment, output, user_to_passed, post_type_passed):
    # List of input vars
    #   comment: dict type. the post (original post or its comment, including comment of comment) to be read.
    #   output: dataframe type. output is a dataframe that treat each post (and comment) as a row.
    #   user_to_passed: dict type, {'name', 'role'}.
    #       to pass user information from upper level post (comment) to next level.
    #   post_type_passed: str type. similar to user_to_passed, but to pass post type.

    # read post author of the current level post
    # author = comment['user']['name']
    # author_role = comment['user']['role']
    # author_email = comment['user']['email']
    author = comment['user']

    # create user info dict to pass to next level
    user_to = {'name': None, 'role': None, 'email': None}

    # Inherit post information from the record (row) above.
    # Because comments usually do not have 'type', 'title', 'category', 'subcategory', or 'private'
    for elem in df_col_names[0:-8] :
        if elem not in comment :
            #print("elem not in list")
            #print(len(output))
            comment[elem] = output.iloc[-1, output.columns.get_loc(elem)]

    # If comments exist, author info will be passed to next level posts as user_to.
    if comment['comments'] : # comments is not empty
        user_to = author
        # user_to['name'] = author
        # user_to['role'] = author_role
        # user_to['email'] = author_email
    # else : # comments is empty
        # user_to['name'] = None
        # user_to['role'] = None
        # user_to['email'] = None

    # Answers will be marked as answer type.
    if post_type_passed == 'answer' :
        comment['type'] = post_type_passed

    # Add new record to existing output dataframe.
    # Each new record (row) contains all information related to the post.
    output = output.append({'url': comment['url'],
                    'type': comment['type'],
                    'title': comment['title'],
                    'category': comment['category'],
                    'subcategory': comment['subcategory'],
                    'votes': comment['votes'],
                    'private': comment['private'],
                    'created_at': comment['created_at'],
                    'text': comment['text'],
                    'user_from': author['name'],
                    'user_from_role': author['role'],
                    'user_from_email': author['email'],
                    'user_to': user_to_passed['name'],
                    'user_to_role': user_to_passed['role'],
                    'user_to_email': user_to_passed['email']}, ignore_index=True)

    # Go over all comments/answers
    replies = comment['comments']
    for reply in replies:
        # recursive part
        output = read_comment(reply, output, user_to, None)

        # replier = reply['user']['name']
        # replier_role = reply['user']['role']
        # replier_email = reply['user']['email']
        #
        # output.iloc[-1, output.columns.get_loc('user_from')] = replier
        # output.iloc[-1, output.columns.get_loc('user_from_role')] = replier_role
        # output.iloc[-1, output.columns.get_loc('user_from_email')] = replier_email

    if 'answers' in comment :
        answers = comment['answers']
        user_to = author
        # user_to['name'] = author
        # user_to['role'] = author_role
        # user_to['email'] = author_email
        for answer in answers:
            output = read_comment(answer, output, user_to, 'answer')

            # replier = answer['user']['name']
            # replier_role = answer['user']['role']
            # replier_email = answer['user']['email']
            # output.iloc[-1, output.columns.get_loc('user_from')] = replier
            # output.iloc[-1, output.columns.get_loc('user_from_role')] = replier_role
            # output.iloc[-1, output.columns.get_loc('user_from_email')] = replier_email

    return output

## 5, Prepare dfbeta1_master.csv for SNA
For all 8 courses, read .json files and create a dataframe to store all posts/comments. Then read .csv files for the
user information, and add the enrollment track to the dataframe.

In [7]:
# define path of folder where all data files are stored.
data_dir = './Beta1_Data/'
# define tags of the courses.
course_list = ['BAMM101', 'BAMM102', 'BAMM103', 'BAMM104', 'CSMM101', 'CSMM102', 'CSMM103', 'CSMM104']

# define the final output dataframe colnames.
df_col_names = ['url', 'type', 'title', 'category', 'subcategory', 'votes', 'private', 'created_at',
                'text', 'user_from', 'user_from_role', 'user_from_email', 'user_to', 'user_to_role', 'user_to_email']

json_file = '' # json file name
user_file = '' # user file name
append_switch = False # indicator of whether to create a new csv file or append data to existing file.

# for each course (course tag), find the json_file and user_file to prepare final output dataframe.
for keyword in course_list :
    for fname in os.listdir(data_dir) :
        if keyword in fname :
            if '.json' in fname : json_file = fname
            if '.csv' in fname : user_file = fname

    print("Processing course", keyword, "\n.json file is", json_file, "\n.csv file is ", user_file)

    # create output_df as a dataframe to store all records (posts and comments) in the json file.
    output_df = json_to_csv(json_file)

    # add course_id and thread_id to output_df
    output_df['course_id'] = keyword
    # create thread_id based on url.
    output_df['thread_id'] = output_df['url']
    output_df['thread_id'] = output_df['url'].str.split("?").str.get(0).str.split("courses/").str.get(1)

    # link user information from user_file: we need the enrollment track for each user_from and user_to
    output_df['user_from_track'] = ""
    output_df['user_to_track'] = ""

    # read user information
    user_raw = pd.read_csv(str(data_dir + user_file), sep = ',')
    user_df = user_raw[["Username", "Email", "Enrollment Track"]]

    # link two datasets. two approaches could be use: 1, pd.merge; 2, for each post/comment, find user_from/user_to in
    # the user_df, and pass the Enrollment Track value to output_df.
    # Please note that the user names in two datasets do not always match. So, user's email address is used as the keys.

    # Approach 1: pd.merge. This is quicker.
    track_df = pd.merge(output_df['user_from_email'], user_df, left_on="user_from_email", right_on="Email", how="left")
    output_df['user_from_track'] = track_df['Enrollment Track']

    track_df = pd.merge(output_df['user_to_email'], user_df, left_on="user_to_email", right_on="Email", how="left")
    output_df['user_to_track'] = track_df['Enrollment Track']

    # # Approach 2: for each post/comment, find user_from/user_to in the user_df, and pass the Enrollment Track value to output_df.
    # for i in range(0, len(output_df)) :
    #     current_record = output_df.iloc[i].copy()
    #
    #     enroll_track = user_df.loc[user_df['Email'] == current_record['user_from_email'], 'Enrollment Track']
    #     if len(enroll_track) != 0 :
    #         output_df.iloc[i, output_df.columns.get_loc('user_from_track')] = enroll_track.iloc[0]
    #
    #     enroll_track = user_df.loc[user_df['Email'] == current_record['user_to_email'], 'Enrollment Track']
    #     if len(enroll_track) != 0 :
    #         output_df.iloc[i, output_df.columns.get_loc('user_to_track')] = enroll_track.iloc[0]

    # Save to csv file.
    if not append_switch : # first course, so new file will be created.
        output_df.to_csv("./dfbeta1_master.csv", sep=',', index=False, mode='w')

    if append_switch : # following courses, so output_df will be appended.
        output_df.to_csv("./dfbeta1_master.csv", sep=',', index=False, mode='a', header=False)
        
    append_switch = True




Processing course BAMM101 
.json file is BAMM101_ Analytics in Python 3T2019 discussion threads.json 
.csv file is  ColumbiaX_BAMM101x_3T2019_grade_report_2019-12-29-2124.csv
Processing course BAMM102 
.json file is BAMM102_ Data, Model and Decisions 3T2019 discussion threads.json 
.csv file is  ColumbiaX_BAMM102x_3T2019_grade_report_2019-12-29-2122.csv
Processing course BAMM103 
.json file is BAMM103_ Demand and Supply Analytics 3T2019 discussion threads.json 
.csv file is  ColumbiaX_BAMM103x_3T2019_grade_report_2019-12-29-2121.csv
Processing course BAMM104 
.json file is BAMM104_ Marketing Analytics 3T2019 discussion threads.json 
.csv file is  ColumbiaX_BAMM104x_3T2019_grade_report_2019-12-29-2127.csv
Processing course CSMM101 
.json file is CSMM101_ Artificial Intelligence 3T2019 discussion threads.json 
.csv file is  ColumbiaX_CSMM101x_3T2019_grade_report_2019-12-29-0248.csv
Processing course CSMM102 
.json file is CSMM102_ Machine Learning 3T2019 discussion threads.json 
.csv fil